# Note book projet 8 : Matthieu Gschwend

## Initialisation

In [1]:
# init

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1677447548249_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Import des librairies

In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Chemins d'accès à S3

In [4]:
PATH = 's3://oc-projet8-test'
PATH_Data = PATH+'/sample_data_train'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://oc-projet8-test
PATH_Data:   s3://oc-projet8-test/sample_data_train
PATH_Result: s3://oc-projet8-test/Results

### Lecture des fichier 

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://oc-projet8-t...|2023-01-19 11:08:18| 59856|[FF D8 FF E0 00 1...|
|s3://oc-projet8-t...|2023-01-19 11:08:18| 57198|[FF D8 FF E0 00 1...|
|s3://oc-projet8-t...|2023-01-19 11:08:18| 56926|[FF D8 FF E0 00 1...|
|s3://oc-projet8-t...|2023-01-19 11:08:18| 56797|[FF D8 FF E0 00 1...|
|s3://oc-projet8-t...|2023-01-19 11:08:18| 53788|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

### Création d'une catégorie Label 

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))

print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------------+--------------------+
|path                                                                  |label               |
+----------------------------------------------------------------------+--------------------+
|s3://oc-projet8-test/sample_data_train/apple_crimson_snow_1/r0_40.jpg |apple_crimson_snow_1|
|s3://oc-projet8-test/sample_data_train/apple_crimson_snow_1/r0_186.jpg|apple_crimson_snow_1|
|s3://oc-projet8-test/sample_data_train/apple_crimson_snow_1/r0_180.jpg|apple_crimson_snow_1|
|s3://oc-projet8-test/sample_data_train/apple_crimson_snow_1/r0_190.jpg|apple_crimson_snow_1|
|s3://oc-projet8-test/sample_data_train/apple_crimson_snow_1/r0_4.jpg  |apple_crimson_snow_1|
+----------------------------------------------------------------------+--------------------+
only showing top 5 rows

None

In [9]:
images.groupBy('label').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|               label|count|
+--------------------+-----+
|apple_crimson_snow_1|    6|
|             apple_6|    6|
|    apple_braeburn_1|    6|
|      apple_golden_1|    6|
+--------------------+-----+

## Transfert Learning

In [10]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 0s 0us/step

**Amputation de la dernière couche du modèle**

In [12]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On envoie les poids sur les workers une bonne fois pour toute**

In [19]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
brodcast_weights

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On indique à spark qu'il faut utiliser les poids broadcastés**

In [23]:
new_model.set_weights(brodcast_weights.value)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Appercu du modèle**

In [24]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

**Construction de la fonction featurize_udf pour extraire les features des images**

In [25]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

# udf user define fonction (indispensable pour spark) pour adapter la fonction featurize_series pour l'usage pyspark

#@pandas_udf(transforme ça : 'array<float>' , en ça : PandasUDFType.SCALAR_ITER)
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)
        
        #comme un générateur en python permet de renvoyer des resultats durant le processus. 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

### Calcul des features

In [27]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
features_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                path|               label|            features|
+--------------------+--------------------+--------------------+
|s3://oc-projet8-t...|    apple_braeburn_1|[0.6803765, 0.864...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.1459774, 0.096...|
|s3://oc-projet8-t...|    apple_braeburn_1|[1.1213963, 0.454...|
|s3://oc-projet8-t...|    apple_braeburn_1|[0.754042, 1.0267...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.11529742, 0.10...|
|s3://oc-projet8-t...|      apple_golden_1|[0.0, 0.21774046,...|
|s3://oc-projet8-t...|      apple_golden_1|[0.0, 0.56523496,...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.12837058, 0.11...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.09394203, 0.12...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.07546184, 0.12...|
|s3://oc-projet8-t...|             apple_6|[0.7758167, 1.758...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.44648364, 0.22...|
|s3://oc-projet8-t...|   

**Ecriture des features sur S3**  
Pour ne pas avoir à refaire cette étape la prochaine fois

In [32]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Réduction de dimension

In [41]:
df = spark.read.parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                path|               label|            features|
+--------------------+--------------------+--------------------+
|s3://oc-projet8-t...|    apple_braeburn_1|[0.8570935, 1.110...|
|s3://oc-projet8-t...|             apple_6|[0.7758167, 1.758...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.44648364, 0.22...|
|s3://oc-projet8-t...|      apple_golden_1|[0.012956005, 0.5...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.07546184, 0.12...|
|s3://oc-projet8-t...|      apple_golden_1|[3.6582781E-4, 0....|
+--------------------+--------------------+--------------------+
only showing top 6 rows

**Conversion de "features" en dense vector**

In [36]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
convertUDF = udf(lambda vs: Vectors.dense(vs), VectorUDT())

df_convert = df.select(col("path"), \
          col("label"), \
    convertUDF(col("features")).alias("features") )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_convert.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                path|               label|            features|
+--------------------+--------------------+--------------------+
|s3://oc-projet8-t...|    apple_braeburn_1|[0.85709351301193...|
|s3://oc-projet8-t...|             apple_6|[0.77581667900085...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.44648364186286...|
|s3://oc-projet8-t...|      apple_golden_1|[0.01295600458979...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.07546184211969...|
|s3://oc-projet8-t...|      apple_golden_1|[3.65827814675867...|
+--------------------+--------------------+--------------------+
only showing top 6 rows

### Execution d'un exemple de PCA

In [39]:
n_components = 5
pca = PCA(
    k = n_components, 
    inputCol = 'features', 
    outputCol = 'pcaFeatures'
).fit(df_convert)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_pca = pca.transform(df_convert)
print('Explained Variance Ratio', pca.explainedVariance.toArray())
df_pca.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explained Variance Ratio [0.4452804  0.1849302  0.08855127 0.06108586 0.0370677 ]
+--------------------+--------------------+--------------------+--------------------+
|                path|               label|            features|         pcaFeatures|
+--------------------+--------------------+--------------------+--------------------+
|s3://oc-projet8-t...|    apple_braeburn_1|[0.85709351301193...|[-18.382577847867...|
|s3://oc-projet8-t...|             apple_6|[0.77581667900085...|[-2.2894336739582...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.44648364186286...|[-11.562520495492...|
|s3://oc-projet8-t...|      apple_golden_1|[0.01295600458979...|[6.48727849976521...|
|s3://oc-projet8-t...|apple_crimson_snow_1|[0.07546184211969...|[-11.234026533381...|
|s3://oc-projet8-t...|      apple_golden_1|[3.65827814675867...|[6.79889438573909...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 6 rows

**Variance expliquée**

In [43]:
pca.explainedVariance.sum()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8169154433993073

In [44]:
pca.explainedVariance

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([0.4453, 0.1849, 0.0886, 0.0611, 0.0371])

## Sauvegarde des résultats  

In [46]:
from pyspark.sql.types import StringType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
for col in df_pca.columns:
     df_pca = df_pca.withColumn(col, df_pca[col].cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_pca

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, label: string, features: string, pcaFeatures: string]

In [51]:
df_pca.repartition(1).write.csv("s3://oc-projet8-test/Resultsfinal2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…